In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

In [12]:
!pip install pyspark==3.0.2

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list,udf,lit,explode,split,col,lower,trim,regexp_replace, substring, concat_ws, concat

In [17]:
spark = SparkSession.builder.master("local[*]").appName("ICP2_Sreenivas_Ratnala").enableHiveSupport().getOrCreate()

In [18]:
from google.colab import files
files.upload()

Saving ICP-2.txt.docx to ICP-2.txt.docx


{'ICP-2.txt.docx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x002\x91oWf\x01\x00\x00\xa5\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [20]:
data = spark.read.text('ICP-2.txt.docx')

In [21]:
data.show()

+---------------------+
|                value|
+---------------------+
| PK     ! 2�o...|
| O����X� �*��V$:�...|
| /P��I��~7$��i�...|
| �L�+�v����dc...|
| ��	��^�[��x ����1...|
| f?��3-���޲]�Tꓸ2�j...|
| ���z���ŉ�,	�	�/�...|
| �O�])I��ҽ0��vvr�...|
| �̷��@�}�w�4��...|
|                     |
| <\�l��������q���...|
| �@8�a���52g +�...|
|T�I㰍�B���3�C=K...|
| ��r�E������Qj��...|
| �E/ν@�}��{a�n�I*...|
|                    ~|
|             "��z�6͎K|
| A����e��'�~[�_>...|
|              ?���n|
| _7LP��XY���6��...|
+---------------------+
only showing top 20 rows



In [22]:
data_words = data.select(explode(split(regexp_replace(trim(lower(col("value"))), "[\$#,’.\t\n-]", ""), "\s+")).alias("value"))


In [23]:
data_words.show()


+--------------------+
|               value|
+--------------------+
|pk     ! 2�o...|
|�(�           ...|
|��{�1��ښl�w%�=...|
|o����x� �*��v:�...|
|         /p��i��~7|
|��i��j&b0z�du�t�o...|
|�l�+�v����dc...|
|������{�v�4p����w...|
|        _rels/rels|
|�(�           ...|
|@���ѽq���n/c���...|
|����^�[��x ����1x�p�|
|���f��i)ʃ�y���...|
|i")��c���qu���~3�...|
|f?��3���޲]�tꓸ2�j)...|
|���z���ŉ���/�|f\...|
|�o�])i��ҽ0��vvr�...|
|�̷��@�}�w�4��...|
|               ��9��|
|��!q��z��6h���...|
+--------------------+
only showing top 20 rows



In [24]:
first_char = data_words.select((substring("value",1,1)).alias("first_character"),col("value").alias("word"))


In [25]:
first_char.show()


+---------------+--------------------+
|first_character|                word|
+---------------+--------------------+
|              p|pk     ! 2�o...|
|              �|�(�           ...|
|              �|��{�1��ښl�w%�=...|
|              o|o����x� �*��v:�...|
|              |         /p��i��~7|
|              �|��i��j&b0z�du�t�o...|
|              �|�l�+�v����dc...|
|              �|������{�v�4p����w...|
|               |        _rels/rels|
|              �|�(�           ...|
|              @|@���ѽq���n/c���...|
|              �|����^�[��x ����1x�p�|
|              �|���f��i)ʃ�y���...|
|              i|i")��c���qu���~3�...|
|              f|f?��3���޲]�tꓸ2�j)...|
|              |���z���ŉ���/�|f\...|
|              �|�o�])i��ҽ0��vvr�...|
|              �|�̷��@�}�w�4��...|
|              �|               ��9��|
|              �|��!q��z��6h���...|
+---------------+--------------------+
only showing top 20 rows



In [26]:
first_char.filter(first_char.first_character=="t").show()


+---------------+--------------------+
|first_character|                word|
+---------------+--------------------+
|              t|t�i㰍�b���3�c=k�;|
|              t|                   t|
+---------------+--------------------+



In [27]:
words_group = first_char.groupBy("first_character").agg(concat_ws(",", collect_list("word")).alias("grouped_words"))


In [28]:
words_group.show()


+---------------+---------------------+
|first_character|        grouped_words|
+---------------+---------------------+
|              7| 7b�?������]t���)...|
|              ~| ~,~����ԇ�t�z�...|
|              | ��,"�b��p�z��...|
|              ;|                ;���|
|              l| l8��h5�4���?^��d...|
|              ϻ|                   ϻ�|
|              x| xumx�/�;���uw...|
|              �| �(�           ...|
|              |              %��8��|
|              ˵|                    ˵|
|              <| <\�l��������q���...|
|              ]| ]���>=qn���m)�...|
|              g| gj�dӵe�c�ca�gv��[...|
|              | ��c�8�rԡ���f�u...|
|              | �m�����y�2�f(r...|
|              8| 8�vb���k(ɞ��n�...|
|              +| +)�f20p�h�'d�...|
|              ܋| ܋;�v�����8�:n...|
|              |��]���d3�e欬/�3�...|
|              !| !9p�d�"��jbf�o��...|
+---------------+---------------------+
only showing top 20 rows



In [29]:
results = words_group.orderBy("first_character").select(concat(col("first_character"),lit(", "),col("grouped_words")).alias("Result"))


In [30]:
results.show()


+---------------------+
|               Result|
+---------------------+
|             , ,,,,,,|
|  ,  _rels/rels,...|
| , jf�y���~gb,...|
| , lp>��?�w��h...|
| , ����p,�!g�sz...|
| , %͵ԉ�*y��dέ...|
| , /p��i��~7, ...|
| ,  @0;��1x�!ٞ...|
| , �s�?�d�q�q�ji�|
|           , %��8��|
| , �m�����y�2�...|
| , e�����ն ��d��...|
| , ��c�8�rԡ���f...|
| , ��,"�b��p�...|
|, ��]���d3�e欬/...|
| , �5v�0u���ޘ��j...|
| , �w��u��,� ...|
| , ���z���ŉ���/�...|
| , b/�4���v���...|
| , ��ˋ׳�{��q�...|
+---------------------+
only showing top 20 rows



In [31]:
results.coalesce(1).write.format("text").option("header", "false").mode("append").save("output.txt")
